In [1]:
import ast
import joblib
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Layer, Conv1D, MaxPooling1D, Flatten, LSTM
from tensorflow.keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from scipy.interpolate import interp1d
import contextlib
import io

import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

pd.set_option('display.float_format', lambda x: '%.1f' % x)
np.set_printoptions(precision=2)

In [ ]:
def split_dataframe(df, train_size=0.7, val_size=0.2):
    """
    Découpe un DataFrame temporel en trois parties : train, val, test,
    en respectant l'ordre chronologique.

    Args:
        df (pd.DataFrame): Données à découper (indexé ou non par le temps)
        train_size (float): Proportion pour l'ensemble d'entraînement
        val_size (float): Proportion pour la validation

    Returns:
        df_train, df_val, df_test (DataFrames)
    """
    df = df.copy()
    n = len(df)
    train_end = int(n * train_size)
    val_end = int(n * (train_size + val_size))

    df_train = df.iloc[:train_end]
    df_val = df.iloc[train_end:val_end]
    df_test = df.iloc[val_end:]

    print(f"Train size : {len(df_train)}")
    print(f"Val size : {len(df_val)}")
    print(f"Test size : {len(df_test)}")

    return df_train.copy(), df_val.copy(), df_test.copy()

In [ ]:
def evaluate_model(y_true, y_pred):
    """
    Calcule et affiche les métriques MAE, RMSE et R² entre les vraies valeurs et les prédictions.

    Args:
        y_true (array-like): Valeurs réelles.
        y_pred (array-like): Valeurs prédites.
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f'MAE: {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'R2 Score: {r2:.4f}')

    return {"MAE": mae, "RMSE": rmse, "R2 Score": r2}

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

    """
This function prepares the input features and target values in the format required for training a recurrent neural network (RNN) or LSTM model for sequential prediction tasks. It creates sequences of input features and their corresponding target values, which can be fed into the model during training.

    - X: This parameter represents the input features, typically a pandas DataFrame containing multiple time-series variables such as temperature, humidity, etc.
    - y: This parameter represents the target values, which are typically the values we want to predict based on the input features.
    - time_steps: This parameter defines the length of each sequence. It determines how many data points from the past will be used to predict the next data point. For example, if time_steps is set to 3, the function will create sequences of three consecutive data points as input features and the next data point as the target value.
    """

In [ ]:
def train_temperature_model(X, y, epochs=100, batch_size=32, validation_split=0.2):
    """
    Construit, compile et entraîne un modèle Keras pour prédire 24 températures horaires.

    Paramètres
    ----------
    X : ndarray shape (n_samples, n_features)
        Tableau des caractéristiques d’entrée (ex. Tmin, Tmoy, Tmax, jour_annee, mois).
    y : ndarray shape (n_samples, n_outputs)
        Tableau des cibles (24 températures horaires).
    epochs : int, optional (default=100)
        Nombre d’époques d’entraînement.
    batch_size : int, optional (default=32)
        Taille du batch.
    validation_split : float, optional (default=0.2)
        Fraction des données réservée à la validation.

    Retour
    ------
    model : keras.Model
        Le modèle entraîné.
    history : keras.callbacks.History
        L’historique d’entraînement (pertes et métriques).
    """
    input_dim = X.shape[1]
    output_dim = y.shape[1]

    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        # régression, donc pas d’activation finale
        Dense(output_dim)
    ])

    model.compile(optimizer=Adam(),
                  loss='mse',
                  metrics=['mae'])

    history = model.fit(X, y,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_split=validation_split,
                        verbose=1)

    return model, history

In [ ]:
def load_data_drias(path_fichier_excel):
    """
    Charge un fichier Excel DRIAS avec une colonne 'Date' au format '%d/%m/%Y',
    et retourne un DataFrame avec la date en index.

    Args:
        path_fichier_excel (str): Chemin vers le fichier Excel.

    Returns:
        pd.DataFrame: Données DRIAS avec l'index daté.
    """
    df = pd.read_excel(path_fichier_excel)
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df.set_index('Date', inplace=True)
    display(df)

    return df

In [ ]:
def predict_and_inverse_transform(model, X, y, target_transformer):
    """
    Effectue la prédiction avec le modèle donné et applique l'inverse de la transformation 
    sur les prédictions et les vraies valeurs cibles.

    Args:
        model: Modèle entraîné (ex. BiLSTM).
        X_val: Données d'entrée de validation.
        y_val: Vraies valeurs cibles de validation.
        target_transformer: Transformateur utilisé pour normaliser les cibles (ex. MinMaxScaler).

    Returns:
        Tuple (y_pred_original_scale, y_true_original_scale)
    """
    # Prédiction
    y_pred = model.predict(X)

    # Inversion de la transformation des prédictions
    y_pred_inv = target_transformer.inverse_transform(y_pred)

    # Reshape puis inversion de la transformation des vraies valeurs
    y = y.reshape(-1, 1)
    y_val_inv = target_transformer.inverse_transform(y)

    return y_pred_inv, y_val_inv

In [ ]:
def convertir_q_en_rh(q_kgkg, temperature_C, pression_hPa=1013.25):
    """
    Convertit une série d'humidité spécifique (kg/kg) et de température (°C)
    en humidité relative (%) en supposant une pression constante.

    Paramètres :
        q_kgkg : pd.Series ou np.array d'humidité spécifique (kg/kg)
        temperature_C : pd.Series ou np.array de température (°C)
        pression_hPa : pression atmosphérique en hPa (par défaut = 1013.25)

    Retour :
        pd.Series ou np.array d'humidité relative (%) — même type que l'entrée
    """
    q = np.asarray(q_kgkg)
    T = np.asarray(temperature_C)

    # Pression partielle de vapeur d'eau (e) [hPa]
    e = (q * pression_hPa) / (0.622 + 0.378 * q)

    # Pression de vapeur saturante (e_s) [hPa] — formule de Tetens
    e_s = 6.112 * np.exp((17.67 * T) / (T + 243.5))

    # Humidité relative RH [%]
    RH = 100 * e / e_s
    RH = np.clip(RH, 0, 100)

    # Renvoyer dans le même format que l'entrée
    if isinstance(q_kgkg, pd.Series):
        return pd.Series(RH, index=q_kgkg.index, name='RH_%')
    else:
        return RH


def calculate_relative_humidity(tmean, huss):
    """
    Calcule l'humidité relative (hr) en % à partir des données :
    - tasmin : température minimale journalière à 2m (°C)
    - tasmax : température maximale journalière à 2m (°C)
    - huss   : humidité spécifique à 2m (g/kg)

    Toutes les entrées peuvent être scalaires, des tableaux NumPy ou des colonnes pandas.
    """

    # 2. Pression de vapeur de saturation psat (en hPa)
    psat = np.where(
        tmean < 0,
        10 ** (2.7862 + (9.7561 * tmean) / (272.67 + tmean)),
        10 ** (2.7862 + (7.5526 * tmean) / (239.21 + tmean))
    )

    # 3. Calcul de l’humidité relative en pourcentage
    hr = (huss / 0.622) / psat * 10000
    hr = np.minimum(hr, 100)  # Limiter à 100 %

    return hr

In [ ]:
def convertir_humidite_kgkg_en_gkg(serie_kgkg):
    """
    Convertit une série Pandas d'humidité spécifique de kg/kg en g/kg.

    Paramètres :
    - serie_kgkg : pd.Series contenant des valeurs en kg/kg

    Retour :
    - pd.Series contenant les valeurs converties en g/kg
    """
    return serie_kgkg * 1000

In [ ]:
def extract_24h_values(group):
    if len(group) == 24:
        return group.values
    else:
        print(group)

##### **Dataset DRIAS RCP**

In [ ]:
data_rcp_2_6 = load_data_drias(
    'Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp2.6_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

In [ ]:
data_rcp_4_5 = load_data_drias(
    'Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp4.5_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

In [ ]:
data_rcp_8_5 = load_data_drias(
    'Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp8.5_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

##### **Dataset Prunay**
🌦️ Description des colonnes - Station météo Reims - Prunay

In [ ]:
data_prunay = pd.read_csv('Atmo/reims_hourly_2007_2025.csv')
data_prunay

In [ ]:
data_prunay['time'] = pd.to_datetime(
    data_prunay['time'], format='%Y-%m-%d %H:%M:%S')
data_prunay.set_index('time', inplace=True)
data_prunay

In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = data_rcp_2_6.index.intersection(data_prunay.index)
data_rcp_2_6_ = data_rcp_2_6.loc[common_index_data_rcp_2_6]

In [ ]:
# récupérer les index communs data rcp4.5
common_index_data_rcp_4_5 = data_rcp_4_5.index.intersection(data_prunay.index)
data_rcp_4_5_ = data_rcp_4_5.loc[common_index_data_rcp_4_5]

In [ ]:
# récupérer les index communs data rcp8.5
common_index_data_rcp_8_5 = data_rcp_8_5.index.intersection(data_prunay.index)
data_rcp_8_5_ = data_rcp_8_5.loc[common_index_data_rcp_8_5]

In [ ]:
data_prunay = data_prunay[["temperature_2m",
                           "relative_humidity_2m", "precipitation"]].copy()
data_prunay.rename(columns={"relative_humidity_2m": "prunay_RH"}, inplace=True)
data_prunay.rename(
    columns={"temperature_2m": "prunay_Temperature"}, inplace=True)
data_prunay.rename(
    columns={"precipitation": "prunay_Precipitation"}, inplace=True)
data_prunay = data_prunay[~data_prunay.index.duplicated(keep='first')]
data_prunay.dropna(inplace=True)
RH = convertir_q_en_rh(data_rcp_2_6["hussAdjust"], data_rcp_2_6["tasAdjust"])
data_rcp_2_6["rcp_RH"] = RH

In [ ]:
data_prunay

In [ ]:
data_prunay_hourly_predict = data_prunay.copy()

In [ ]:
data_rcp_2_6

In [ ]:
data_prunay_hourly_predict

In [ ]:
data_prunay_hourly_temp_predict = data_prunay_hourly_predict["prunay_Temperature"].copy(
)

In [ ]:
data_prunay_hourly_hr_predict = data_prunay_hourly_predict.copy()

In [ ]:
data_prunay_hourly_temp_predict = data_prunay_hourly_temp_predict.resample(
    'D').agg(['min', 'mean', 'max'])

In [ ]:
data_prunay_hourly_hr_predict = data_prunay_hourly_hr_predict.resample('D').agg({
    'prunay_RH': ['mean'],
    'prunay_Temperature': ['min', 'mean', 'max'],
    'prunay_Precipitation': ['mean']
})

In [ ]:
data_prunay_hourly_hr_predict.columns = ['{}_{}'.format(
    col[0], col[1]) for col in data_prunay_hourly_hr_predict.columns]

In [ ]:
data_prunay_hourly_temp_predict["day_of_year"] = data_prunay_hourly_temp_predict.index.dayofyear
data_prunay_hourly_temp_predict["month"] = data_prunay_hourly_temp_predict.index.month
data_prunay_hourly_temp_predict["hour"] = data_prunay_hourly_temp_predict.index.hour

In [ ]:
data_prunay_hourly_hr_predict["day_of_year"] = data_prunay_hourly_hr_predict.index.dayofyear
data_prunay_hourly_hr_predict["month"] = data_prunay_hourly_hr_predict.index.month
data_prunay_hourly_hr_predict["hour"] = data_prunay_hourly_hr_predict.index.hour

In [ ]:
data_prunay_hourly_hr_predict

In [ ]:
data_prunay_hourly_temp_predict

In [ ]:
y_temp = data_prunay["prunay_Temperature"].resample(
    'D').apply(extract_24h_values)

In [ ]:
y_rh = data_prunay["prunay_RH"].resample('D').apply(extract_24h_values)

In [ ]:
data_prunay_hourly_temp_predict["y"] = y_temp

In [ ]:
data_prunay_hourly_hr_predict["y"] = y_rh

In [ ]:
data_prunay_hourly_temp_predict.dropna(inplace=True)

In [ ]:
data_prunay_hourly_hr_predict.dropna(inplace=True)

In [ ]:
data_prunay_hourly_temp_predict

In [ ]:
data_prunay_hourly_hr_predict

In [ ]:
def preprocess_data(df, features, pred):
    # Transformation cyclique pour les caractéristiques temporelles
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # Cible - conversion en matrice 2D (n_jours × 24_heures)
    y = np.vstack(df['y'].values)

    # Normalisation
    if pred == "temp":
        X_scaler = StandardScaler()
        y_scaler = StandardScaler()

    if pred == "hr":
        X_scaler = StandardScaler()
        y_scaler = MinMaxScaler()

    X = X_scaler.fit_transform(df[features])
    y = y_scaler.fit_transform(y)

    return X, y, X_scaler, y_scaler, features

In [ ]:
class TanhRange(Layer):
    def __init__(self, min_val=0, max_val=100, **kwargs):
        super().__init__(**kwargs)
        self.min_val = min_val
        self.max_val = max_val

    def call(self, inputs):
        tanh = tf.tanh(inputs)  # [-1, 1]
        scaled = (tanh + 1) / 2  # [0, 1]
        return scaled * (self.max_val - self.min_val) + self.min_val

    def get_config(self):
        config = super().get_config()
        config.update({
            "min_val": self.min_val,
            "max_val": self.max_val
        })
        return config


def build_model(input_dim, pred):
    model = Sequential([
        Dense(256, activation='relu',
              kernel_initializer='he_normal',
              kernel_regularizer=l2(0.001),
              input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.3),

        Dense(128, activation='relu',
              kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),

        Dense(64, activation='relu'),
        Dense(24),  # 24 sorties pour les données horaires
    ])

    if pred == "hr":
        # Couche personnalisée ajoutée ici
        model.add(TanhRange(min_val=0, max_val=100))

    optimizer = Adam(learning_rate=0.001, clipvalue=0.5)
    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mae'])

    model.summary()
    return model

In [ ]:
# Entraînement avec validation temporelle
def train_model(X, y, pred):
    # Séparation temporelle (plus adaptée pour les données chronologiques)
    tscv = TimeSeriesSplit(n_splits=5)

    best_val_loss = float('inf')
    best_model = None

    for fold, (train_index, val_index) in enumerate(tscv.split(X)):
        print(f"\nEntraînement fold {fold+1}/{tscv.get_n_splits()}")
        print(
            f"Taille train: {len(train_index)}, validation: {len(val_index)}")

        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        model = build_model(X.shape[1], pred)

        callbacks = [
            EarlyStopping(patience=10, restore_best_weights=True,
                          monitor='val_loss'),
            ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
        ]

        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=100,
            batch_size=16,
            callbacks=callbacks,
            verbose=1
        )

        # Sauvegarder le meilleur modèle
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model
            print(f"Nouveau meilleur modèle avec val_loss = {val_loss:.4f}")

    return best_model

In [ ]:
def evaluate_model(model, X_test, y_test, y_scaler, pred):
    # Dénormalisation
    y_test_actual = y_scaler.inverse_transform(y_test)
    y_pred = y_scaler.inverse_transform(model.predict(X_test))

    # Calcul MAE par heure
    hourly_mae = np.mean(np.abs(y_pred - y_test_actual), axis=0)

    # Visualisation
    plt.figure(figsize=(14, 6))

    # MAE par heure
    plt.subplot(1, 2, 1)
    plt.bar(range(24), hourly_mae, color='skyblue')
    plt.title('Mean Absolute Error (MAE) per hour')
    plt.xlabel('Hour of the day')
    plt.ylabel('MAE')
    plt.xticks(range(24), [f'{h:02d}h' for h in range(24)], rotation=45)
    plt.grid(alpha=0.3)

    # Comparaison sur un échantillon
    sample_idx = np.random.randint(len(X_test))
    plt.subplot(1, 2, 2)
    plt.plot(y_test_actual[sample_idx], 'o-', label='true value', linewidth=2)
    plt.plot(y_pred[sample_idx], 's-', label='Prediction', linewidth=1.5)
    plt.title(f'Prediction vs True value - Random Day')
    plt.xlabel('Hours')
    plt.ylabel("Temp (°C)" if pred == "temp" else "HR (%)")
    plt.legend()
    plt.xticks(range(0, 24, 3), [f'{h:02d}h' for h in range(0, 24, 3)])
    plt.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig('model_evaluation.png', dpi=300)
    plt.show()

    if pred == "temp":
        print(f"\nModel performance:")
        print(f"Mean MAE: {np.mean(hourly_mae):.4f}°C")
        print(
            f"Max MAE (at {np.argmax(hourly_mae):02d}h): {np.max(hourly_mae):.4f}°C")
        print(
            f"Min MAE (at {np.argmin(hourly_mae):02d}h): {np.min(hourly_mae):.4f}°C")
    if pred == "hr":
        print(f"\nModel performance:")
        print(f"Mean MAE: {np.mean(hourly_mae):.4f}%")
        print(
            f"Max MAE (at {np.argmax(hourly_mae):02d}h): {np.max(hourly_mae):.4f}%")
        print(
            f"Min MAE (at {np.argmin(hourly_mae):02d}h): {np.min(hourly_mae):.4f}%")

    return hourly_mae

In [ ]:
def train_and_evaluate_model_temporal_split(X, y, y_scaler, train_model, evaluate_model, test_size=0.2, pred="temp"):
    """
    Entraîne et évalue un modèle de prédiction de température avec validation temporelle

    Args:
        X (np.array): Features normalisées
        y (np.array): Cibles normalisées 
        y_scaler (StandardScaler): Scaler pour la dénormalisation des cibles
        test_size (float): Proportion des données à utiliser pour le test (0.0-1.0)

    Returns:
        model (keras.Model): Modèle entraîné
        hourly_mae (np.array): MAE par heure sur l'ensemble de test
    """
    # Séparation train/test temporelle
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    print("\nSéparation des données:")
    print(f"Train: {X_train.shape[0]} échantillons")
    print(f"Test: {X_test.shape[0]} échantillons")

    # Entraînement
    print("\nDébut de l'entraînement du modèle...")
    model = train_model(X_train, y_train, pred)

    # Évaluation
    print("\nÉvaluation sur l'ensemble de test...")
    hourly_mae = evaluate_model(model, X_test, y_test, y_scaler, pred)

    return model, hourly_mae

In [ ]:
X_temp, y_temp, X_scaler_temp, y_scaler_temp, features_temp = preprocess_data(
    data_prunay_hourly_temp_predict, ['min', 'max', 'mean', 'day_sin', 'day_cos', 'month_sin', 'month_cos'], pred="temp")

In [ ]:
X_hr, y_hr, X_scaler_hr, y_scaler_hr, features_hr = preprocess_data(data_prunay_hourly_hr_predict, [
                                                                    'prunay_RH_mean', 'prunay_Temperature_min', 'prunay_Temperature_max', 'prunay_Temperature_mean', 'prunay_Precipitation_mean', 'day_sin', 'day_cos', 'month_sin', 'month_cos'], pred="hr")

In [ ]:
model_temp, hourly_mae_temp = train_and_evaluate_model_temporal_split(
    X_temp, y_temp, y_scaler_temp, train_model=train_model, evaluate_model=evaluate_model, pred="temp")

In [ ]:
model_hr, hourly_mae_hr = train_and_evaluate_model_temporal_split(X_hr, y_hr, y_scaler_hr, train_model=train_model, evaluate_model=evaluate_model, pred="hr")

In [ ]:
def compute_day_stats(temps_array):
    return {
        'min_temp': float(np.min(temps_array)),
        'max_temp': float(np.max(temps_array)),
        'mean_temp': float(np.mean(temps_array)),
        'day_of_year': temps_array.index.dayofyear[0],
        'month': temps_array.index.month[0]
    }

In [ ]:
# Fonction de prédiction
def predict_hourly_temperatures(min_temp, max_temp, mean_temp, day_of_year, month, model, X_scaler, y_scaler):
    # Préparation de l'input
    day_sin = np.sin(2 * np.pi * day_of_year / 365.25)
    day_cos = np.cos(2 * np.pi * day_of_year / 365.25)
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)

    input_data = np.array([[min_temp, max_temp, mean_temp,
                            day_sin, day_cos, month_sin, month_cos]])

    # Transformation
    scaled_input = X_scaler.transform(input_data)
    prediction = model.predict(scaled_input)
    return y_scaler.inverse_transform(prediction)[0]


# Exemple de prédiction
print("\nExemple de prédiction...")

new_day = {
    'min_temp': 0.1,
    'max_temp': 3.0,
    'mean_temp': 1.5125,
    'day_of_year': 32,
    'month': 2
}

predictions = predict_hourly_temperatures(new_day["min_temp"], new_day["max_temp"], new_day["mean_temp"],
                                          new_day["day_of_year"], new_day["month"], model_temp, X_scaler_temp, y_scaler_temp)

# Visualisation de la prédiction
plt.figure(figsize=(10, 6))
plt.plot(predictions, 'o-', color='darkorange', linewidth=2)
# plt.plot(data_prunay.loc["2015-02-01"]["prunay_Temperature"].values, 'o-', color='red', linewidth=2)
plt.title(
    f"Prédiction de température - Jour {new_day['day_of_year']} (Mois {new_day['month']})")
plt.xlabel('Heure de la journée')
plt.ylabel('Température (°C)')
plt.xticks(range(24), [f'{h:02d}h' for h in range(24)], rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('prediction_example.png', dpi=300)
plt.show()

In [ ]:
compute_day_stats(data_prunay.loc["2015-02-01"]["prunay_Temperature"])
data_prunay.loc["2015-02-01"]["prunay_Temperature"].index.dayofyear[0]

In [ ]:
data_prunay.loc["2015-02-01"]["prunay_Temperature"].plot()

In [ ]:
compute_day_stats(data_prunay.loc["2015-02-01"]["prunay_RH"])

In [ ]:
compute_day_stats(data_prunay.loc["2015-02-01"]["prunay_Precipitation"])

In [ ]:
predict_hourly_temperatures(new_day["min_temp"], new_day["max_temp"], new_day["mean_temp"],
                            new_day["day_of_year"], new_day["month"], model_temp, X_scaler_temp, y_scaler_temp)

In [ ]:
predictions

In [ ]:
def predict_hourly_hr(prunay_RH_mean, prunay_Temperature_min, prunay_Temperature_max, prunay_Temperature_mean, prunay_Precipitation_mean, day_of_year, month, model, X_scaler, y_scaler):
    # Préparation de l'input
    day_sin = np.sin(2 * np.pi * day_of_year / 365.25)
    day_cos = np.cos(2 * np.pi * day_of_year / 365.25)
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)

    input_data = np.array([[prunay_RH_mean, prunay_Temperature_min, prunay_Temperature_max, prunay_Temperature_mean, prunay_Precipitation_mean,
                            day_sin, day_cos, month_sin, month_cos]])

    # Transformation
    scaled_input = X_scaler.transform(input_data)
    prediction = model.predict(scaled_input)
    return y_scaler.inverse_transform(prediction)[0]


# Exemple de prédiction
print("\nExemple de prédiction...")


new_day = {
    # humidité moyenne journalière (%)
    'prunay_RH_mean': 81.29166666666667,
    'prunay_Temperature_min': 1.0,
    'prunay_Temperature_max': 5.9,
    'prunay_Temperature_mean': 3.591,
    'prunay_Precipitation_mean': 0.0625,
    'day_of_year': 325,
    'month': 11,
}

predictions = predict_hourly_hr(new_day["prunay_RH_mean"], new_day["prunay_Temperature_min"], new_day["prunay_Temperature_max"],
                                new_day["prunay_Temperature_mean"], new_day["prunay_Precipitation_mean"], new_day["day_of_year"], new_day["month"], model_hr, X_scaler_hr, y_scaler_hr)

# Visualisation
plt.figure(figsize=(10, 6))
plt.plot(predictions, 'o-', color='teal', linewidth=2)
plt.title(
    f"Prédiction de l'humidité - Jour {new_day['day_of_year']} (Mois {new_day['month']})")
plt.xlabel('Heure de la journée')
plt.ylabel("Humidité (%)")
plt.xticks(range(24), [f'{h:02d}h' for h in range(24)], rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('prediction_humidity_example.png', dpi=300)
plt.show()

In [ ]:
data_prunay.loc["2007-02-01"].resample('D').agg(['min', 'mean', 'max'])

In [ ]:
data_prunay.loc["2024-11-20"]["prunay_RH"].plot()

In [ ]:
compute_day_stats(data_prunay.loc["2024-11-20"]["prunay_RH"])

In [ ]:
compute_day_stats(data_prunay.loc["2024-11-20"]["prunay_Temperature"])

In [ ]:
compute_day_stats(data_prunay.loc["2024-11-20"]["prunay_Precipitation"])

In [ ]:
data_prunay.loc["2008-07-01"]["prunay_RH"].min()

In [ ]:
predictions.min()

In [ ]:
data_prunay.loc["2008-07-01"]["prunay_RH"].max()

In [ ]:
predictions.max()

In [ ]:
data_prunay.loc["2007-02-01"]["prunay_RH"].mean()

In [ ]:
predictions.mean()

In [ ]:
compute_day_stats(data_prunay.loc["2007-07-01"]["prunay_Temperature"])

In [ ]:
compute_day_stats(data_prunay.loc["2007-07-01"]["prunay_RH"])

In [ ]:
compute_day_stats(data_prunay.loc["2007-07-01"]["prunay_Precipitation"])

In [ ]:
def predict_from_dataframe_temp(df_input, model, X_scaler, y_scaler):

    # Copie pour éviter d'altérer le DataFrame d'origine
    df = df_input.copy()

    # Ajout des features cycliques
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # Features
    features = ['min', 'max', 'mean',
                'day_sin', 'day_cos', 'month_sin', 'month_cos']

    # Normalisation
    X = X_scaler.transform(df[features])

    # Prédiction
    y_pred_scaled = model.predict(X)

    # Retour à l'échelle originale
    y_pred = y_scaler.inverse_transform(y_pred_scaled)

    # Ajout dans le DataFrame sous forme de listes
    df['y_pred'] = [list(row) for row in y_pred]

    return df

In [ ]:
def predict_from_dataframe_hr(df_input, model, X_scaler, y_scaler):

    # Copie pour éviter d'altérer le DataFrame d'origine
    df = df_input.copy()

    # Ajout des features cycliques
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # Features
    features = ['prunay_RH_mean', 'prunay_Temperature_min', 'prunay_Temperature_max', 'prunay_Temperature_mean', 'prunay_Precipitation_mean',
                'day_sin', 'day_cos', 'month_sin', 'month_cos']

    # Normalisation
    X = X_scaler.transform(df[features])

    # Prédiction
    y_pred_scaled = model.predict(X)

    # Retour à l'échelle originale
    y_pred = y_scaler.inverse_transform(y_pred_scaled)

    # Ajout dans le DataFrame sous forme de listes
    df['y_pred'] = [list(row) for row in y_pred]

    return df

In [ ]:
data_rcp_2_6predict_hourly_temp = data_rcp_2_6[[
    'tasminAdjust', 'tasmaxAdjust', 'tasAdjust']].copy()

In [ ]:
data_rcp_2_6predict_hourly_hr = data_rcp_2_6[[
    'rcp_RH', 'tasminAdjust', 'tasmaxAdjust', 'tasAdjust', 'prtotAdjust']].copy()

In [ ]:
data_rcp_2_6predict_hourly_temp

In [ ]:
data_rcp_2_6predict_hourly_hr

In [ ]:
data_rcp_2_6predict_hourly_temp.rename(
    columns={"tasminAdjust": "min", "tasmaxAdjust": "max", "tasAdjust": "mean"}, inplace=True)

In [ ]:
data_rcp_2_6predict_hourly_hr.rename(columns={"rcp_RH": "prunay_RH_mean", "tasminAdjust": "prunay_Temperature_min",
                                     "tasmaxAdjust": "prunay_Temperature_max", "tasAdjust": "prunay_Temperature_mean", "prtotAdjust": "prunay_Precipitation_mean"}, inplace=True)

In [ ]:
data_rcp_2_6predict_hourly_temp["day_of_year"] = data_rcp_2_6predict_hourly_temp.index.dayofyear
data_rcp_2_6predict_hourly_temp["month"] = data_rcp_2_6predict_hourly_temp.index.month

In [ ]:
data_rcp_2_6predict_hourly_hr["day_of_year"] = data_rcp_2_6predict_hourly_hr.index.dayofyear
data_rcp_2_6predict_hourly_hr["month"] = data_rcp_2_6predict_hourly_hr.index.month

In [ ]:
data_rcp_2_6predict_hourly_temp = predict_from_dataframe_temp(
    data_rcp_2_6predict_hourly_temp, model=model_temp, X_scaler=X_scaler_temp, y_scaler=y_scaler_temp)

In [ ]:
data_rcp_2_6predict_hourly_hr = predict_from_dataframe_hr(
    data_rcp_2_6predict_hourly_hr, model=model_hr, X_scaler=X_scaler_hr, y_scaler=y_scaler_hr)

In [ ]:
data_rcp_2_6predict_hourly_hr

In [ ]:
data_rcp_2_6predict_hourly_temp

In [ ]:
df_hourly_temp = data_rcp_2_6predict_hourly_temp[["y_pred"]].copy()

In [ ]:
df_hourly_hr = data_rcp_2_6predict_hourly_hr[["y_pred"]].copy()

In [ ]:
def daily_to_hourly(df, value_col="y"):
    """
    Transforme un DataFrame journalier (1 ligne = 1 jour, 1 cellule = 24 valeurs)
    en DataFrame horaire (1 ligne = 1 heure).

    Paramètres :
    ------------
    df : pd.DataFrame
        DataFrame journalier, index = dates, colonne = listes de 24 valeurs horaires.
    value_col : str
        Nom de la colonne contenant les valeurs horaires.

    Retour :
    --------
    df_hourly : pd.DataFrame
        DataFrame horaire, index = datetime (jour + heure), colonnes : ['y']
    """
    df = df.copy()

    # Assurer que l’index est de type datetime
    df.index = pd.to_datetime(df.index)

    # Répéter chaque ligne 24 fois
    df_expanded = df.loc[df.index.repeat(24)].copy()

    # Aplatir les valeurs horaires
    df_expanded[value_col] = df[value_col].explode().values

    # Ajouter les heures (0 à 23)
    df_expanded["hour"] = list(range(24)) * len(df)

    # Créer l’index horaire
    df_expanded["datetime"] = df_expanded.index + \
        pd.to_timedelta(df_expanded["hour"], unit='h')

    # Finaliser
    df_hourly = df_expanded[["datetime", value_col]
                            ].set_index("datetime").sort_index()

    return df_hourly

In [ ]:
df_hourly_temp = daily_to_hourly(df_hourly_temp, "y_pred")

In [ ]:
df_hourly_hr = daily_to_hourly(df_hourly_hr, "y_pred")

In [ ]:
df_hourly_temp

In [ ]:
df_hourly_hr

In [ ]:
def pourcentages_par_intervalles(array):
    array = np.array(array)
    total = len(array)

    pct_0_30 = np.sum((array >= 0) & (array < 30)) / total * 100
    pct_30_50 = np.sum((array >= 30) & (array < 50)) / total * 100
    pct_50_70 = np.sum((array >= 50) & (array < 70)) / total * 100
    pct_70_90 = np.sum((array >= 70) & (array < 90)) / total * 100
    pct_90_100 = np.sum((array >= 90) & (array < 100)) / total * 100
    pct_eq_100 = np.sum(array == 100) / total * 100
    pct_sup_100 = np.sum(array > 100) / total * 100

    return {
        '0–30%': pct_0_30,
        '30–50%': pct_30_50,
        '50–70%': pct_50_70,
        '70–90%': pct_70_90,
        '90–100%': pct_90_100,
        '==100%': pct_eq_100,
        '>100%': pct_sup_100
    }

In [ ]:
plt.figure(figsize=(15, 8))
sns.set_style('darkgrid')

correlation_matrix = round(data_rcp_2_6.select_dtypes('number').corr(), 2)

correlation_with_trgt = correlation_matrix['rcp_RH'].sort_values(
    ascending=False)

ax = sns.barplot(x=correlation_with_trgt.index,
                 y=correlation_with_trgt, palette='viridis')

plt.title('Correlation with meantemp', size=20)
plt.xlabel('Features')
plt.ylabel('Correlation')

for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.xticks(rotation=45, ha='right')

In [ ]:
pourcentages_par_intervalles(df_hourly_hr)

In [ ]:
pourcentages_par_intervalles(data_prunay["prunay_RH"])

In [ ]:
(df_hourly_hr > 100).sum()

In [ ]:
df_hourly_hr[df_hourly_hr >= 100] = 100

In [ ]:
df_hourly_hr

In [ ]:
df_hourly_temp

In [ ]:
df_hourly_temp.rename(columns={"y_pred": "rcp_2_6_Temperature"}, inplace=True)
df_hourly_hr.rename(columns={"y_pred": "rcp_2_6_RH"}, inplace=True)

In [ ]:
data_rcp_2_6_hourly_temp_hr = pd.concat([df_hourly_temp, df_hourly_hr], axis=1)

In [ ]:
data_rcp_2_6_hourly_temp_hr.loc["2024-07"]

In [ ]:
data_prunay[data_prunay["prunay_Temperature"] < 1]

In [ ]:
data_prunay.loc["2024-07", ["prunay_Temperature", "prunay_RH"]]

In [ ]:
data_rcp_2_6_hourly_temp_hr

In [ ]:
data_rcp_2_6_hourly_temp_hr

In [ ]:
data_prunay

#### **LSTM**

Bidirectional LSTMs are an extension of traditional LSTMs that can improve model performance on sequence classification problems.

In problems where all timesteps of the input sequence are available, Bidirectional LSTMs train two instead of one LSTMs on the input sequence. The first on the input sequence as-is and the second on a reversed copy of the input sequence. This can provide additional context to the network and result in faster and even fuller learning on the problem.


In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = data_rcp_2_6_hourly_temp_hr.index.intersection(
    data_prunay.index)
data_rcp_2_6_hourly_temp_hr_ = data_rcp_2_6_hourly_temp_hr.loc[common_index_data_rcp_2_6]

In [ ]:
data_rcp_2_6_hourly_temp_hr_[
    "prunay_Temperature"] = data_prunay["prunay_Temperature"]

In [ ]:
data_rcp_2_6_hourly_temp_hr_["prunay_RH"] = data_prunay["prunay_RH"]

In [ ]:
data_rcp_2_6_hourly_temp_hr_predict_temp = data_rcp_2_6_hourly_temp_hr_[
    ['rcp_2_6_Temperature', 'rcp_2_6_RH', 'prunay_Temperature']].copy()

In [ ]:
data_rcp_2_6_hourly_temp_hr_predict_hr = data_rcp_2_6_hourly_temp_hr_[
    ['rcp_2_6_Temperature', 'rcp_2_6_RH', 'prunay_RH']].copy()

In [ ]:
data_rcp_2_6_hourly_temp_hr_predict_hr

In [ ]:
# Paramètres du modèle
WINDOW_SIZE = 48 # Fenêtre de 24 heures (ajustable)
PREDICTION_HORIZON = 1  # Prédire l'heure suivante
TEST_SIZE = 0.2  # 20% pour le test

# Features et target
FEATURES = ['rcp_2_6_Temperature', 'rcp_2_6_RH']
TARGET = 'prunay_RH'

In [ ]:
# Normalisation des données
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Scaling des features
X_scaled = scaler_X.fit_transform(
    data_rcp_2_6_hourly_temp_hr_predict_hr[FEATURES])
# Scaling de la target
y_scaled = scaler_y.fit_transform(
    data_rcp_2_6_hourly_temp_hr_predict_hr[[TARGET]])

# Création des séquences pour LSTM


def create_sequences(X, y, window_size, prediction_horizon):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size - prediction_horizon + 1):
        X_seq.append(X[i:(i + window_size)])
        y_seq.append(y[i + window_size + prediction_horizon - 1])
    return np.array(X_seq), np.array(y_seq)


X_sequences, y_sequences = create_sequences(
    X_scaled, y_scaled, WINDOW_SIZE, PREDICTION_HORIZON)

print(f"Forme des séquences X: {X_sequences.shape}")
print(f"Forme des séquences y: {y_sequences.shape}")

In [ ]:
# Division train/test
split_idx = int(len(X_sequences) * (1 - TEST_SIZE))

X_train = X_sequences[:split_idx]
X_test = X_sequences[split_idx:]
y_train = y_sequences[:split_idx]
y_test = y_sequences[split_idx:]

print(f"Train shapes: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"Test shapes: X_test {X_test.shape}, y_test {y_test.shape}")

In [ ]:
def build_lstm_model(input_shape):
    model = Sequential([
    LSTM(128, return_sequences=True, input_shape=input_shape, 
         kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    
    LSTM(64, return_sequences=True, 
         kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    LSTM(32, return_sequences=False,
         kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(32, activation='relu'),
    Dense(1)
])

    model.compile(
        # optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )

    return model


# Construction du modèle
input_shape = (X_train.shape[1], X_train.shape[2])  # (window_size, n_features)
model = build_lstm_model(input_shape)

print(model.summary())

In [ ]:

callbacks = [
    EarlyStopping(patience=25, restore_best_weights=True),
    ReduceLROnPlateau(patience=10, factor=0.5)
]

# Entraînement
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Prédictions
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Récupération des dates pour le test set
dates = data_rcp_2_6_hourly_temp_hr_predict_hr.index[WINDOW_SIZE +
                                                       PREDICTION_HORIZON - 1:]
test_dates = dates[split_idx:split_idx + len(X_test)]

# Métriques
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f"MAE: {mae:.2f} %")
print(f"RMSE: {rmse:.2f} %")

# Visualisation des résultats
plt.figure(figsize=(40, 5))

# Courbes de prédiction
plt.subplot(1, 2, 1)
plt.plot(test_dates[:5000], y_true[:5000], label='Trues Values', alpha=0.7)
plt.plot(test_dates[:5000], y_pred[:5000], label='Predictions', alpha=0.7)
plt.title('Predictions vs Trues Values')
plt.xlabel('Dates')
plt.ylabel('Humidity (%)')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
temp_rcp_2_6_data_bias_correction = data_rcp_2_6_hourly_temp_hr.copy()

In [ ]:
def predict_dataframe_temperature(df, temp_col, rh_col, model, scaler_X, scaler_y, window_size):
    temps = df[temp_col].values
    rh = df[rh_col].values
    X = np.column_stack([temps, rh])

    # mise à l'échelle
    X_scaled = scaler_X.transform(X)

    # construction des séquences
    sequences = []
    for i in range(len(X_scaled) - window_size + 1):
        sequences.append(X_scaled[i:i + window_size])

    sequences = np.array(sequences)

    # prédiction
    y_scaled = model.predict(sequences, verbose=0)
    y_pred = scaler_y.inverse_transform(y_scaled).flatten()

    # alignement des prédictions avec df
    preds = np.full(len(df), np.nan)
    preds[window_size - 1:] = y_pred
    df['predicted_hr'] = preds
    return df


# Utilisation avec votre DataFrame
temp_rcp_2_6_data_bias_correction_ = predict_dataframe_temperature(
    df=temp_rcp_2_6_data_bias_correction,
    temp_col="rcp_2_6_Temperature",
    rh_col="rcp_2_6_RH",
    model=model,
    scaler_X=scaler_X,
    scaler_y=scaler_y,
    window_size=WINDOW_SIZE
)

In [2]:
rcp_2_6_data_bias_correction_Temp = pd.read_csv('temp_rcp_2_6_data_bias_correction_csv_Temp.csv')

In [3]:
rcp_2_6_data_bias_correction_HR = pd.read_csv('temp_rcp_2_6_data_bias_correction_HR.csv')

In [4]:
rcp_2_6_data_bias_correction_Temp.set_index('datetime', inplace=True)

In [5]:
rcp_2_6_data_bias_correction_HR.set_index('datetime', inplace=True)

In [6]:
rcp_2_6_data_bias_correction_HR

,rcp_2_6_Temperature,rcp_2_6_RH,predicted_temp,predicted_hr
datetime,,,,
2006-01-01 00:00:00,-1.2,92.8,NaN,NaN
2006-01-01 01:00:00,-1.5,93.7,NaN,NaN
2006-01-01 02:00:00,-1.7,94.4,NaN,NaN
2006-01-01 03:00:00,-2.0,95.0,NaN,NaN
2006-01-01 04:00:00,-2.3,95.3,NaN,NaN
...,...,...,...,...
2100-12-31 19:00:00,1.7,85.2,84.5,84.5
2100-12-31 20:00:00,1.5,86.2,85.8,85.8
2100-12-31 21:00:00,1.2,88.8,86.5,86.5


In [7]:
rcp_2_6_data_bias_correction_Temp_HR = pd.DataFrame(index=rcp_2_6_data_bias_correction_Temp.index.copy())

In [8]:
rcp_2_6_data_bias_correction_Temp_HR["predicted_temp"] = rcp_2_6_data_bias_correction_Temp["predicted_temp"].copy()

In [9]:
rcp_2_6_data_bias_correction_Temp_HR["predicted_hr"] = rcp_2_6_data_bias_correction_HR["predicted_hr"].copy()

In [10]:
rcp_2_6_data_bias_correction_Temp_HR.index = pd.to_datetime(rcp_2_6_data_bias_correction_Temp_HR.index)

In [11]:
rcp_2_6_data_bias_correction_Temp_HR

,predicted_temp,predicted_hr
datetime,,
2006-01-01 00:00:00,NaN,NaN
2006-01-01 01:00:00,NaN,NaN
2006-01-01 02:00:00,NaN,NaN
2006-01-01 03:00:00,NaN,NaN
2006-01-01 04:00:00,NaN,NaN
...,...,...
2100-12-31 19:00:00,5.6,84.5
2100-12-31 20:00:00,5.1,85.8
2100-12-31 21:00:00,4.8,86.5


In [12]:
data_basilique = pd.read_excel('Releves/DATA_processing_iButton_2018_2019.xlsx')

In [13]:
data_basilique.set_index('Date Heure', inplace=True)

In [14]:
# data_basilique.index = data_basilique.index.round('h')

In [15]:
data_basilique = data_basilique[['N2OTemp', 'S2OHR']].copy()

In [16]:
# récupérer les index communs data rcp8.5
common_index_data_basilique_rcp_2_6_data_bias_correction_Temp_HR = rcp_2_6_data_bias_correction_Temp_HR.index.intersection(data_basilique.index)
rcp_2_6_data_bias_correction_Temp_HR_ = rcp_2_6_data_bias_correction_Temp_HR.loc[common_index_data_basilique_rcp_2_6_data_bias_correction_Temp_HR]

In [17]:
rcp_2_6_data_bias_correction_Temp_HR_["N2OTemp"] = data_basilique["N2OTemp"].copy()
rcp_2_6_data_bias_correction_Temp_HR_["S2OHR"] = data_basilique["S2OHR"].copy()

In [18]:
rcp_2_6_data_bias_correction_Temp_HR_

,predicted_temp,predicted_hr,N2OTemp,S2OHR
Date Heure,,,,
2018-01-01 00:00:01,NaN,NaN,9.7,82.8
2018-01-01 01:00:01,NaN,NaN,9.0,92.2
2018-01-01 02:00:01,NaN,NaN,8.9,80.8
2018-01-01 03:00:01,NaN,NaN,8.8,79.4
2018-01-01 04:00:01,NaN,NaN,8.0,87.6
...,...,...,...,...
2019-12-31 19:38:01,NaN,NaN,2.5,91.9
2019-12-31 20:38:01,NaN,NaN,1.8,95.3
2019-12-31 21:38:01,NaN,NaN,1.3,94.4


In [ ]:
data_basilique